### For SCDA simulator

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
import hcipy
import scipy
from scipy.interpolate import griddata
import glob
import numpy as np
from PIL import Image
from astropy.io import fits
import astropy.units as u
import pandas as pd
import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib.gridspec as gridspec
from pastis.temporal_analysis.close_loop_analysis import req_closedloop_calc_batch
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
import pastis.util as util  
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.scda_telescopes import HexRingAPLC

### User define parameters

In [ ]:
# Define input parameters
optics_dir = os.path.join(util.find_repo_location(), 'data', 'SCDA')
sampling = 4
num_rings = 1
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 

### Create directory to save your plots

In [ ]:
data_dir = CONFIG_PASTIS.get('local', 'local_data_path')
repo_dir = os.path.join(util.find_repo_location(package='pastis'))

overall_dir = util.create_data_path(data_dir, telescope='7hexaplc_design')
resDir = os.path.join(overall_dir, 'matrix_numerical')
print(resDir)

# Create necessary directories if they don't exist yet
os.makedirs(resDir, exist_ok=True)

### Instantiate HeXRingAPLC

In [ ]:
tel = HexRingAPLC(optics_dir, num_rings, sampling)

### Create deformable mirror

In [ ]:
filepath = CONFIG_PASTIS.get('LUVOIR', 'harris_data_path')
pad_orientation = np.pi/2*np.ones(tel.nseg)
tel.create_segmented_harris_mirror(filepath,pad_orientation, thermal=True,mechanical=False,other=False) 
tel.harris_sm 

#### get/redefine number of dm actuators and num of thermal modes

In [ ]:
num_actuators = tel.harris_sm.num_actuators
num_modes = 5 #n

### Calculate contrast floor for coronagraphic psf without aberration

In [ ]:
tel.harris_sm.flatten()
unaberrated_coro_psf, ref = tel.calc_psf(ref=True, display_intermediate=False, norm_one_photon=True)
norm = np.max(ref)

dh_intensity = (unaberrated_coro_psf / norm) * tel.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(tel.dh_mask != 0)])
print(f'contrast floor: {contrast_floor}')

In [ ]:
ideal_coro_psf = np.reshape(unaberrated_coro_psf, [115,115])
ideal_psf = np.reshape(ref, [115,115])
ind_max = np.unravel_index(np.argmax(np.abs(ideal_psf), axis=None), np.abs(ideal_psf).shape)

fits.writeto((os.path.join(resDir, 'ideal_psf.fits')), np.abs(ideal_psf))

plt.figure(figsize=(22,9))
plt.subplot(1,2,1)
plt.title("Coronagraphic PSF", fontsize=25)
plt.imshow(np.log10(np.abs(ideal_coro_psf)), cmap='magma')
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
plt.colorbar()

plt.subplot(1,2,2)
plt.title("PSF without Coronagraph", fontsize=25)
plt.imshow(np.log10(np.abs(ideal_psf)), cmap ='magma')
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='out')
plt.colorbar()
plt.tight_layout()
plt.savefig(os.path.join(resDir, 'ideal_coro_psf.png'))

### Calculate PASTIS matrix

In [ ]:
nonaberrated_coro_psf, ref, efield = tel.calc_psf(ref=True, display_intermediate=True, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
print('Generating the E-fields for harris modes in science plane')
print(f'Calibration aberration used: {nm_aber} m')

start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for pp in range(0, num_actuators):
    print(f'Working on mode {pp}/{num_actuators}')
    
    # Apply calibration aberration to used mode
    harris_actuators = np.zeros(num_actuators)
    harris_actuators[pp] = 1*(nm_aber)/2 
    tel.harris_sm.actuators  = harris_actuators
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = tel.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
pastis_matrix = np.zeros([num_actuators, num_actuators])   # create empty matrix

for i in range(0, num_actuators):
    for j in range(0, num_actuators):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref ) * np.conj(focus_fieldS[j].electric_field -Efield_ref ))
        dh_test = (test / norm) * tel.dh_mask
        contrast = np.mean(dh_test[np.where(tel.dh_mask != 0)])
        pastis_matrix[i, j] = contrast

In [ ]:
clist = [(0.1, 0.6, 1.0), (0.05, 0.05, 0.05), (0.8, 0.5, 0.1)]
blue_orange_divergent = LinearSegmentedColormap.from_list("custom_blue_orange", clist)

plt.figure(figsize=(10,8))
norm_mat = TwoSlopeNorm(vcenter=0)
plt.imshow(pastis_matrix, cmap=blue_orange_divergent,norm = norm_mat)
plt.title(r"PASTIS matrix $M$", fontsize=20)
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("Mode Index",fontsize=20)
plt.tick_params(labelsize=15)
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label(r"in units of ${contrast\ per\ {nm}^2}$",fontsize =15)
plt.tight_layout()

plt.savefig(os.path.join(resDir, 'pastis_matrix.png'))

### Save PASTIS matrix and poked efields 

In [ ]:
filename_matrix = 'PASTISmatrix_n_harris_' + str(num_actuators)
hcipy.write_fits(pastis_matrix, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_Re_matrix_n_harris_' + str(num_actuators)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))


filename_matrix = 'EFIELD_Im_matrix_n_harris_' + str(num_actuators)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

end_time = time.time()
print('Runtime for harris modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

### Eigenvalues and Eigenmodes of PASTIS matrix

In [ ]:
evals, evecs = np.linalg.eig(pastis_matrix)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(sorted_evals)
plt.semilogy()
plt.xlabel('Mode Index',fontsize=20)
plt.ylabel('Eigenvalues (in units of contrast/'r'wave$^2$)',fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,
                labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',direction='in')
plt.tick_params(axis='both',which='major',length=10, width=2)

plt.savefig(os.path.join(resDir, 'eigenvalues.png'))

In [ ]:
emodes = []
eunit = 1e-9
for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    harris_coeffs = eunit*sorted_evecs[:, mode]/2
    tel.harris_sm.actuators = harris_coeffs
    wf_harris_sm = tel.harris_sm(tel.wf_aper)
    emodes.append(wf_harris_sm.phase)

In [ ]:
plt.figure(figsize = (20,15))
for i in range (1, 36):
    ax = plt.subplot(5,7,i)
    hcipy.imshow_field(emodes[i-1])
    plt.colorbar()

In [ ]:
c_target_log = -10
c_target = 10**(c_target_log)

In [ ]:
mu_map_harris = np.sqrt(((c_target) / (num_actuators)) / (np.diag(pastis_matrix)))
np.savetxt(os.path.join(resDir, 'mu_map_harris_%s.csv'%c_target), mu_map_harris, delimiter=',')

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Modal constraints to achieve a dark hole contrast of "r"$10^{%d}$"%c_target_log, fontsize=30)
plt.ylabel("Weight per mode (in units of nm)",fontsize =20)
plt.xlabel("Mode index", fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.plot(mu_map_harris)
plt.tight_layout()
plt.savefig(os.path.join(resDir, 'mu_map_harris_%s.png'%c_target))

In [ ]:
sigma = np.sqrt(((c_target)) / (num_actuators * sorted_evals))
plt.figure(figsize=(15,8))
plt.title("Max mode contribution(s) from the static-contrast target %s and eigen values"%c_target, fontsize=20)
plt.ylabel(r"$\tilde{b}_{p}$ (in units of nm)",fontsize =20)
plt.xlabel("Mode index", fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
plt.plot(sigma)
plt.savefig(os.path.join(resDir, 'sigma_%s.png'%c_target))

In [ ]:
cont_cum_pastis = []
for maxmode in range(sorted_evecs.shape[0]): 
    aber = np.nansum(sorted_evecs[:, :maxmode+1] * sigma[:maxmode+1], axis=1)
    aber *= u.nm
    contrast_matrix = util.pastis_contrast(aber, pastis_matrix) + contrast_floor
    cont_cum_pastis.append(contrast_matrix)
    
plt.figure(figsize=(10,10))
plt.plot(cont_cum_pastis)
plt.xlabel("Mode Index", fontsize =20)
plt.ylabel("Cumulative contrast", fontsize =20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
plt.tight_layout()
plt.savefig(os.path.join(resDir, 'cont_cum_pastis_%s.png'% c_target))

In [ ]:
harris_coeffs_numaps = np.zeros([num_modes, num_actuators])

for qq in range(num_modes):
    harris_coeffs_tmp = np.zeros([num_actuators]) 
    for kk in range(tel.nseg):
        harris_coeffs_tmp[qq+(kk)*num_modes] = mu_map_harris[qq+(kk)*num_modes] #arranged per modal basis
    harris_coeffs_numaps[qq] = harris_coeffs_tmp #arragned into 5 groups of 35 elements and in units of nm

nu_maps = []
for qq in range(num_modes):
    harris_coeffs = harris_coeffs_numaps[qq] #in units of nm
    tel.harris_sm.actuators = harris_coeffs*nm_aber/2 # in units of m
    nu_maps.append(tel.harris_sm.surface) #in units of m, each nu_map is now of the order of 1e-9 m 

### Tolerance map in units of m

In [ ]:
plt.figure(figsize=(40,20))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("Segment Level 1mK Faceplates Silvered", fontsize=30)
hcipy.imshow_field((nu_maps[0])*1e6, cmap = 'RdBu') #nu_map is already in 1e-9 m
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("$\mu m$",fontsize =30)

ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Segment Level 1mK bulk",fontsize=30)
hcipy.imshow_field((nu_maps[1])*1e6,cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("$\mu m$",fontsize =30)

ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Segment Level 1mK gradiant radial",fontsize=30)
hcipy.imshow_field((nu_maps[2])*1e6,cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("$\mu m$",fontsize =30)

ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Segment Level 1mK gradient X lateral",fontsize=30)
hcipy.imshow_field((nu_maps[3])*1e6,cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("$\mu m$",fontsize =30)

ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Segment Level 1mK gradient Z axial",fontsize=30)
hcipy.imshow_field((nu_maps[4])*1e6,cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("$\mu m$",fontsize =30)

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'stat_mu_maps_um_%s.png'%c_target))

### Generate Tolerance maps in units of Kelvin

In [ ]:
harris_coeffs_table = np.zeros([num_modes, tel.nseg])
for qq in range(num_modes):
    for kk in range(tel.nseg):
        harris_coeffs_table[qq,kk] = mu_map_harris[qq+(kk)*num_modes] #numpy ndarray 120 # in units of nm

# Faceplate silvered
tel2 = HexRingAPLC(optics_dir, num_rings, sampling)
tel2.create_segmented_mirror(1) 
tel2.sm.actuators =harris_coeffs_table[0]

# Bulk
tel3 = HexRingAPLC(optics_dir, num_rings, sampling)
tel3.create_segmented_mirror(1)
tel3.sm.actuators = harris_coeffs_table[1]

# Gradient Radial
tel4 = HexRingAPLC(optics_dir, num_rings, sampling)
tel4.create_segmented_mirror(1)
tel4.sm.actuators = harris_coeffs_table[2]

# Gradient X Lateral
tel5 = HexRingAPLC(optics_dir, num_rings, sampling)
tel5.create_segmented_mirror(1)
tel5.sm.actuators = harris_coeffs_table[3]

# Gradient z axial
tel6 = HexRingAPLC(optics_dir, num_rings, sampling)
tel6.create_segmented_mirror(1)
tel6.sm.actuators = harris_coeffs_table[4]

In [ ]:
print("Allowable temperature change (in K)","\n")
print("Faceplate silvered: ",np.sqrt(np.mean(np.square(tel2.sm.actuators))))
print("Bulk: ", np.sqrt(np.mean(np.square(tel3.sm.actuators))))
print("Gradient Radial:",np.sqrt(np.mean(np.square(tel4.sm.actuators))))
print("Gradient X Lateral: ",np.sqrt(np.mean(np.square(tel5.sm.actuators))))
print("Gradient Z axial: ",np.sqrt(np.mean(np.square(tel6.sm.actuators))))

In [ ]:
plt.figure(figsize=(40,20))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("Faceplates Silvered", fontsize=30)
hcipy.imshow_field((tel2.sm.surface), cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("K",fontsize =30)

ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Bulk",fontsize=30)
hcipy.imshow_field((tel3.sm.surface), cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("K",fontsize =30)

ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Gradiant Radial",fontsize=30)
hcipy.imshow_field((tel4.sm.surface), cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("K",fontsize =30)

ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Gradient X lateral",fontsize=30)
hcipy.imshow_field((tel5.sm.surface),cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("K",fontsize =30)

ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Gradient Z axial",fontsize=30)
hcipy.imshow_field((tel6.sm.surface),cmap = 'RdBu')
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("K",fontsize =30)

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'mu_map_K_%s.png'%c_target))

### Temporal Ananlysis

In [ ]:
npup = int(np.sqrt(tel.pupil_grid.x.shape[0]))
nimg = int(np.sqrt(tel.focal_det.x.shape[0]))

sptype = 'A0V' # Put this on config
Vmag = 5.0 # Put this in loop
minlam = 500 * u.nanometer # Put this on config
maxlam = 600 * u.nanometer # Put this on config
dark_current = 0   #put this on config
CIC = 0            #electrons per sec #put this on config
star_flux = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=Vmag, 
                                                      minlam=minlam.value, maxlam=maxlam.value)
Nph = star_flux.value*15**2*np.sum(tel.apodizer**2) / npup**2

In [ ]:
tel.harris_sm.flatten()
nonaberrated_coro_psf, refshit,inter_ref = tel.calc_psf(ref=True, display_intermediate=False, 
                                                        return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

### out-of-band reference electric field seen by out-of-band wavefront sensor

In [ ]:
tel.harris_sm.flatten()

harris_ref2 = tel.calc_out_of_band_wfs(norm_one_photon=True)
harris_ref2_sub_real = hcipy.field.subsample_field(harris_ref2.real, 8, statistic='mean')
harris_ref2_sub_imag = hcipy.field.subsample_field(harris_ref2.imag, 8, statistic='mean')

Efield_ref_OBWFS = (harris_ref2_sub_real + 1j*harris_ref2_sub_imag)

In [ ]:
E0_OBWFS = np.zeros([128*128,1,2])
E0_OBWFS[:,0,0] = Efield_ref_OBWFS.real
E0_OBWFS[:,0,1] = Efield_ref_OBWFS.imag

In [ ]:
plt.figure(figsize = (20,10))
plt.subplot(1,2,1)
hcipy.imshow_field(Efield_ref_OBWFS.real, cmap ='RdBu')
plt.colorbar()
plt.subplot(1,2,2)
hcipy.imshow_field(Efield_ref_OBWFS.imag, cmap ='RdBu')
plt.colorbar()

In [ ]:
E0_coron = np.zeros([nimg*nimg,1,2])
E0_coron[:,0,0] = Efield_ref.real
E0_coron[:,0,1] = Efield_ref.imag

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_n_harris_' + str(num_actuators) + '.fits'
G_harris_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_Im_matrix_n_harris_' + str(num_actuators) + '.fits'
G_harris_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

In [ ]:
G_coron_harris= np.zeros([nimg*nimg,2,num_actuators])
for pp in range(0, num_actuators):
    G_coron_harris[:,0,pp] = G_harris_real[pp] - Efield_ref.real
    G_coron_harris[:,1,pp] = G_harris_imag[pp] - Efield_ref.imag

In [ ]:
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for pp in range(0, num_actuators):
    print(pp)
    harris_actuators = np.zeros(num_actuators)
    harris_actuators[pp] = (nm_aber) / 2
    tel.harris_sm.actuators  = harris_actuators
    harris_meas = tel.calc_out_of_band_wfs(norm_one_photon=True)
    harris_meas_sub_real = hcipy.field.subsample_field(harris_meas.real, 8, statistic='mean')
    harris_meas_sub_imag = hcipy.field.subsample_field(harris_meas.imag, 8, statistic='mean')
    focus_field1 = harris_meas_sub_real + 1j * harris_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_harris_' + str(num_actuators)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_harris_' + str(num_actuators)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_harris_' + str(num_actuators)+'.fits'
G_OBWFS_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_harris_' + str(num_actuators)+'.fits'
G_OBWFS_imag =  fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

In [ ]:
G_OBWFS= np.zeros([128*128, 2, num_actuators])
for pp in range(0, num_actuators):
    G_OBWFS[:,0,pp] = G_OBWFS_real[pp]*8 - Efield_ref_OBWFS.real
    G_OBWFS[:,1,pp] = G_OBWFS_imag[pp]*8 - Efield_ref_OBWFS.imag

In [ ]:
# Running a bunch of tests for time series
flux = Nph
Qharris = np.diag(np.asarray(mu_map_harris**2))

Ntimes = 20
TimeMinus = -10
TimePlus = 5.5 
Nwavescale = 8
WaveScaleMinus = -2
WaveScalePlus = 1
Nflux = 3
fluxPlus = 10
fluxMinus = 0

timeVec = np.logspace(TimeMinus,TimePlus,Ntimes)
WaveVec = np.logspace(WaveScaleMinus,WaveScalePlus,Nwavescale)
fluxVec = np.linspace(fluxMinus,fluxPlus,Nflux) #10**(-fluxVec/2.5)
wavescaleVec = np.logspace(WaveScaleMinus,WaveScalePlus,Nwavescale)

In [ ]:
result_mv =[]

for StarMag in range (2,7,1):
    print('Harris modes closeloop batch estimation, StarMag %f'% StarMag) 
    wavescale = 1.
    niter = 10
    timer1 = time.time()
    for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
        Starfactor = 10**(-StarMag/2.5)
        print(tscale)
        tmp0 = req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*Qharris,niter, tel.dh_mask, norm)    
        tmp1 = tmp0['averaged_hist']
        n_tmp1 = len(tmp1)
        result_mv.append(tmp1[n_tmp1-1])


In [ ]:
texp = np.logspace(TimeMinus, TimePlus, Ntimes)

plt.figure(figsize =(15,10))
plt.plot(texp,result_mv[0:20] - contrast_floor, label=r'$m_{v}=2$')
plt.plot(texp,result_mv[20:40] - contrast_floor, label=r'$m_{v}=3$')
plt.plot(texp,result_mv[40:60] - contrast_floor, label=r'$m_{v}=4$')
plt.plot(texp,result_mv[60:80] - contrast_floor, label=r'$m_{v}=5$')
plt.plot(texp,result_mv[80:100] - contrast_floor, label=r'$m_{v}=6$')
plt.xlabel("$t_{WFS}$ in secs",fontsize=20)
plt.ylabel("$\Delta$ contrast",fontsize=20) 
plt.yscale('log')
plt.xscale('log')
plt.legend(loc = 'lower right',fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, 
                right=True,labelleft=True, labelbottom=True,
                labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',which='minor',length=6, width=2)
plt.grid()
plt.show()
plt.savefig(os.path.join(resDir, 'cont_mv_%s.png'%c_target))

In [ ]:
res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
result_wf_test =[]

for wavescale in range (1,15,2):
    print('recurssive close loop batch estimation and wavescale %f'% wavescale)  
    niter = 10
    timer1 = time.time()
    StarMag = 0.0
    for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
        Starfactor = 10**(-StarMag/2.5)
        print(tscale)
        tmp0 = req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
                                                 dark_current+CIC/tscale, tscale, flux*Starfactor, 1000*wavescale**2*Qharris,
                                                 niter, tel.dh_mask, norm)    
        tmp1 = tmp0['averaged_hist']
        n_tmp1 = len(tmp1)
        result_wf_test.append(tmp1[n_tmp1-1])

In [ ]:
delta_wf = []
for wavescale in range (1,15,2):
    wf = 1e3*np.sqrt(1000*wavescale**2) #converting to pm
    delta_wf.append(wf)
    
texp = np.logspace(TimeMinus, TimePlus, Ntimes)
font = {'family': 'serif','color' : 'black','weight': 'normal','size'  :  20}

plt.figure(figsize =(15,10))
plt.title('Target contrast = %s, Vmag= %s'%(c_target, Vmag),fontdict=font)
plt.plot(texp,result_wf_test[0:20]-contrast_floor, label=r'$\Delta_{wf}= %d\ pm$'%(delta_wf[0]))
plt.plot(texp,result_wf_test[20:40]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[1]))
plt.plot(texp,result_wf_test[40:60]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[2]))
plt.plot(texp,result_wf_test[60:80]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[3]))
plt.plot(texp,result_wf_test[80:100]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[4]))
plt.plot(texp,result_wf_test[100:120]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[5]))
plt.plot(texp,result_wf_test[120:140]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[6]))
plt.xlabel("$t_{WFS}$ in secs",fontsize=20)
plt.ylabel("$\Delta$ contrast",fontsize=20)
plt.yscale('log')
plt.xscale('log')
plt.legend(loc = 'upper center',fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, 
                right=True,labelleft=True, labelbottom=True,
                labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',which='minor',length=6, width=2)
plt.grid()
plt.savefig(os.path.join(resDir, 'cont_wf_%s.png'%c_target))
plt.show()